# Couple Child and SedFlux

PyMT makes it possible to interactively perform a two-way coupling of the Child and SedFlux models, all in Python!

In [ ]:
# Some magic to make plots appear within the notebook
%matplotlib inline
import numpy as np # In case we need to use numpy

In [ ]:
from pymt.components import Sedflux3D, Child

child = Child()
sedflux = Sedflux3D()

child_in, child_dir = child.setup(grid_node_spacing=500.,
                                  grid_x_size=40000.,
                                  grid_y_size=20000.,
                                  run_duration=1e6)
sedflux_in, sedflux_dir = sedflux.setup(river_bed_load_flux=0.,
                                        river_suspended_load_concentration_0=.001,
                                        river_suspended_load_concentration_1=.001,
                                        run_duration=1e6 * 365.)

child.initialize(child_in, dir=child_dir)
sedflux.initialize(sedflux_in, dir=sedflux_dir)

In [ ]:
gid = child.var['land_surface__elevation'].grid
x, y = child.get_grid_x(gid), child.get_grid_y(gid)
z = child.get_value('land_surface__elevation')

In [ ]:
x_shore = 20000.
z[np.where(x > x_shore)] += 100.
z[np.where(x <= x_shore)] -= 100.

child.set_value('land_surface__elevation', z)

In [ ]:
sedflux.set_value('bedrock_surface__elevation', mapfrom=('land_surface__elevation', child))

In [ ]:
child.quick_plot('land_surface__elevation', edgecolors='k', vmin=-200, vmax=200, cmap='BrBG_r')

In [ ]:
sedflux.quick_plot('bedrock_surface__elevation', vmin=-200, vmax=200, cmap='BrBG_r')

In [ ]:
now = child.get_current_time()
times = np.arange(now, now + 100, 1.)
for t in times:
    child.update_until(t, units='years')

    sedflux.set_value('channel_water_sediment~bedload__mass_flow_rate',
                      mapfrom=child)
    sedflux.update_until(t, units='years')
    
    z = child.get_value('land_surface__elevation')
    child.set_value('land_surface__elevation',
                    mapfrom=('land-or-seabed_sediment_surface__elevation', sedflux),
                    nomap=np.where(z>0.))

In [ ]:
child.quick_plot('land_surface__elevation', edgecolors='k', vmin=-100, vmax=-99, cmap='BrBG_r')

In [ ]:
child.quick_plot('channel_water_sediment~bedload__mass_flow_rate', vmax=1e6)

In [ ]:
sedflux.output_var_names

In [ ]:
sedflux.get_current_time(units='year')

In [ ]:
help(sedflux)